In [1]:
import pandas as pd
import numpy as np
from src.utils.elo import Elo

## WIP - I want to calculate ELO ratings and add these as a a features into gameweek data

In [2]:
gameweek_df = pd.read_csv('data/processed/combined_gameweeks.csv')

/opt/homebrew/Caskroom/miniconda/base/envs/ravelin36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (25,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Need to think of a good way to do this? 

In [3]:
gameweek_df.team

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
            ...     
69945      Leicester
69946      Newcastle
69947    Southampton
69948       Brighton
69949       West Ham
Name: team, Length: 69950, dtype: object

In [4]:
pd.set_option('display.max_rows', 100)
gameweek_df.tail().T

,69945,69946,69947,69948,69949
name,Wilfred Ndidi,Matt Ritchie,Nathan Redmond,Mathew Ryan,Ryan Fredericks
assists,0,1,0,0,0
attempted_passes,NaN,NaN,NaN,NaN,NaN
big_chances_created,NaN,NaN,NaN,NaN,NaN
big_chances_missed,NaN,NaN,NaN,NaN,NaN
bonus,0,0,0,0,0
bps,7,22,0,0,0
clean_sheets,0,0,0,0,0
clearances_blocks_interceptions,NaN,NaN,NaN,NaN,NaN
completed_passes,NaN,NaN,NaN,NaN,NaN


In [5]:
teams = pd.read_csv(f'data/Fantasy-Premier-League/data/master_team_list.csv').rename(columns={'team':'opponent_team','team_name':'opponent_name'})

In [6]:
with_opponents = gameweek_df.merge(teams,on=['season','opponent_team'],how='left')

In [12]:
fixtures = with_opponents[['team','opponent_name','season','gw','team_h_score','team_a_score', 'was_home']].loc[with_opponents.team.notna()].drop_duplicates().copy()

In [13]:
fixtures['opp_team'] = 'opp_' + fixtures.opponent_name # so we can merge and update

In [14]:
fixtures['outcome'] =(np.clip(fixtures['team_h_score'] -fixtures['team_a_score'],-1,1) + 1) / 2

In [15]:
elo =Elo(fixtures,k_factor=20, team_h='team',team_a='opp_team')
historic_elo, elo_ratings =elo.process_all_fixtures()

In [16]:
elo_ratings

defaultdict(<function src.utils.elo.Elo.__init__.<locals>.<lambda>()>,
            {'Brighton': 1222.0808735581634,
             'opp_Chelsea': 1218.169713758656,
             'West Ham': 1192.3379908540717,
             'opp_Newcastle': 1201.5214499514286,
             'Sheffield Utd': 1250.6486788780264,
             'opp_Wolves': 1190.2600191571275,
             'Everton': 1181.2590663907713,
             'opp_Spurs': 1159.1854682160565,
             'Fulham': 1152.9248864239214,
             'opp_Arsenal': 1183.5052066906433,
             'Wolves': 1209.6816689516206,
             'opp_Sheffield Utd': 1149.4009647880364,
             'Leeds': 1217.9307702713816,
             'opp_Liverpool': 1224.765573516552,
             'Leicester': 1157.6918732288862,
             'opp_West Brom': 1176.6273208907082,
             'Liverpool': 1174.5167187249867,
             'opp_Leeds': 1181.9158449492106,
             'West Brom': 1223.8329021867153,
             'opp_Leicester': 1242.4221472